In [28]:
import pandas as pd
import numpy as np
import sqlite3
import torch

In [29]:
# Load dataset
con = sqlite3.connect('../paper_data.sqlite3')
papers = pd.read_sql_query("SELECT * FROM paper_data", con)

con.close()

In [30]:
papers.columns

Index(['title', 'description', 'year', 'coverdate', 'publication_name',
       'citation_count', 'subject_areas', 'author_names', 'affiliations',
       'countries', 'filename'],
      dtype='object')

In [31]:
# Search for empty strings in every string column and replace them with NaN
str_cols = papers.select_dtypes(include=['object']).columns

for col in str_cols:
    papers[col] = papers[col].replace('', np.nan)

papers.isna().sum()

title                 1
description         665
year                  0
coverdate             0
publication_name      0
citation_count        0
subject_areas         0
author_names          0
affiliations          0
countries             0
filename              0
dtype: int64

In [32]:
papers.dropna(subset=['title', 'description'], inplace=True)

In [33]:
papers.loc[papers.citation_count == 'None', 'citation_count'] = np.nan
papers['citation_count'] = pd.to_numeric(papers['citation_count'], errors='coerce')

In [34]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
papers['citation_count'] = imputer.fit_transform(papers[['citation_count']])

papers['coverdate'] = pd.to_datetime(papers['coverdate'], errors='coerce')


In [35]:
subject_areas = papers.subject_areas.str.split(',').explode().str.strip().unique().tolist()
instituions = papers.affiliations.str.split(',').explode().str.strip().unique().tolist()

#Get number of authors per paper
papers['author_count'] = papers.author_names.str.split(',').apply(lambda x: len(x) if isinstance(x, list) else 0)
papers['affil_count'] = papers.affiliations.str.split(',').apply(lambda x: len(x) if isinstance(x, list) else 0)
papers['country_count'] = papers.countries.str.split(',').apply(lambda x: len(x) if isinstance(x, list) else 0)

# papers.affiliations.str.split(',').explode().str.strip().value_counts()

In [ ]:
papers.to_csv('papers.csv', index=False)